BREED RECOGNITION MODEL

In [6]:
import os

for root, dirs, files in os.walk("data/train"):
    print(f"{root}: {len(files)} files")

data/train: 0 files


In [7]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

image_size = (224, 224)
batch_size = 32
seed = 42

train_ds = image_dataset_from_directory(
    r"C:\Users\MSII\Desktop\breed recognition data set\Indian_bovine_breeds",
    validation_split=0.2,
    subset="training",
    seed=seed,
    image_size=image_size,
    batch_size=batch_size,
    label_mode="int"
)

val_ds = image_dataset_from_directory(
    r"C:\Users\MSII\Desktop\breed recognition data set\Indian_bovine_breeds",
    validation_split=0.2,
    subset="validation",
    seed=seed,
    image_size=image_size,
    batch_size=batch_size,
    label_mode="int"
)

class_names = train_ds.class_names
print("Detected breeds:", class_names)

Found 5947 files belonging to 41 classes.
Using 4758 files for training.
Found 5947 files belonging to 41 classes.
Using 1189 files for validation.
Detected breeds: ['Alambadi', 'Amritmahal', 'Ayrshire', 'Banni', 'Bargur', 'Bhadawari', 'Brown_Swiss', 'Dangi', 'Deoni', 'Gir', 'Guernsey', 'Hallikar', 'Hariana', 'Holstein_Friesian', 'Jaffrabadi', 'Jersey', 'Kangayam', 'Kankrej', 'Kasargod', 'Kenkatha', 'Kherigarh', 'Khillari', 'Krishna_Valley', 'Malnad_gidda', 'Mehsana', 'Murrah', 'Nagori', 'Nagpuri', 'Nili_Ravi', 'Nimari', 'Ongole', 'Pulikulam', 'Rathi', 'Red_Dane', 'Red_Sindhi', 'Sahiwal', 'Surti', 'Tharparkar', 'Toda', 'Umblachery', 'Vechur']


In [12]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

In [15]:
import tensorflow as tf
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])

In [14]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

Epoch 1/10
149/149 ━━━━━━━━━━━━━━━━━━━━ 110s 705ms/step - accuracy: 0.0801 - loss: 3.6182 - val_accuracy: 0.1194 - val_loss: 3.4663
Epoch 2/10
149/149 ━━━━━━━━━━━━━━━━━━━━ 92s 611ms/step - accuracy: 0.1158 - loss: 3.4414 - val_accuracy: 0.1228 - val_loss: 3.3784
Epoch 3/10
149/149 ━━━━━━━━━━━━━━━━━━━━ 91s 604ms/step - accuracy: 0.1263 - loss: 3.3135 - val_accuracy: 0.1430 - val_loss: 3.3039
Epoch 4/10
149/149 ━━━━━━━━━━━━━━━━━━━━ 246s 2s/step - accuracy: 0.1438 - loss: 3.2438 - val_accuracy: 0.1455 - val_loss: 3.2187
Epoch 5/10
149/149 ━━━━━━━━━━━━━━━━━━━━ 104s 691ms/step - accuracy: 0.1557 - loss: 3.1596 - val_accuracy: 0.1531 - val_loss: 3.2068
Epoch 6/10
149/149 ━━━━━━━━━━━━━━━━━━━━ 98s 652ms/step - accuracy: 0.1667 - loss: 3.0950 - val_accuracy: 0.1615 - val_loss: 3.1078
Epoch 7/10
149/149 ━━━━━━━━━━━━━━━━━━━━ 95s 633ms/step - accuracy: 0.1822 - loss: 3.0232 - val_accuracy: 0.1682 - val_loss: 3.1022
Epoch 8/10
149/149 ━━━━━━━━━━━━━━━━━━━━ 91s 609ms/step - accuracy: 0.1862 - loss: 2

In [20]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomContrast(0.1)
])

train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))

In [22]:
def build_model(base_model_fn, input_shape=(224, 224, 3), num_classes=20):
    base_model = base_model_fn(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False  # Freeze base layers

    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model

In [26]:
num_classes = len(class_names)

In [29]:
inputs = tf.keras.Input(shape=(224, 224, 3))

In [30]:
for images, labels in train_ds.take(1):
    print(images.shape)  # Should be (batch_size, 224, 224, 3)

(32, 224, 224, 3)


In [5]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

NameError: name 'tf' is not defined

In [2]:
def build_model(base_model_fn, input_shape=(224, 224, 3), num_classes=20):
    base_model = base_model_fn(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False

    inputs = tf.keras.Input(shape=input_shape)
    x = base_model(inputs, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = tf.keras.layers.Dense(num_classes, activation="softmax")(x)

    model = tf.keras.Model(inputs, outputs)
    return model

In [4]:
base_model.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

NameError: name 'base_model' is not defined

In [36]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomContrast(0.2),
    tf.keras.layers.RandomBrightness(0.2)
])

train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))

In [1]:
for images, labels in train_ds.take(1):
    print(images.shape)  # Should be (batch_size, height, width, 3)

NameError: name 'train_ds' is not defined

In [38]:
img = tf.image.decode_jpeg(raw_img, channels=3)

NameError: name 'raw_img' is not defined

In [37]:
model = build_model(tf.keras.applications.EfficientNetB3, input_shape=(300, 300, 3), num_classes=num_classes)

43941136/43941136 ━━━━━━━━━━━━━━━━━━━━ 13s 0us/step


ValueError: Shape mismatch in layer #1 (named stem_conv)for weight stem_conv/kernel. Weight expects shape (3, 3, 1, 40). Received saved weight with shape (3, 3, 3, 40)